《Boosting association rule mining in large datasets via Gibbs sampling》 Qian G , Rao C R , Sun X , et al. Boosting association rule mining in large datasets via Gibbs sampling[J]. Proc Natl Acad, U S A, 2016, 113(18):4958.

本文目的是复现上述论文中的4个example。
一些论文笔记：
>+ 1.关联规则挖掘是一个重要的任务
>+ 2.与挖掘数据库中所有关联规则相比较，一个更有趣的问题是挖掘给定结果的情况下最重要的关联规则
>+ 3.论文的关注点是鉴别数据中最重要的关联规则
>+ 4.当前使用的关联规则挖掘方法是基于约束的，导致不能应用于密集型数据，原因：一是关联规则太多导致计算困难；二是基础约束会导致挖掘的规则不完整
>+ 5.论文想要找到的是拥有高置信度的关联规则
>+ 6.定义新的关联规则测度，重要性（importance），表示为关联规则$R_c$的支持度与置信度的增函数
>+ 7.提出一个新的概率分布：
$$P_c(J)=P_c(J => I_c)=\frac{e^{\xi g(J => I_c)}} {\sum_{allJ}{e^{\xi g(J => I_c)}}}$$
>+ 8.引入Gibbs采样的作用是减少规则挖掘的项目空间和得到最优规则出现的频繁项集


1. 一些基本概念
>+ 1.项目空间（item space）：定义一个由$m$个项目组成的集合$ I = \{{I_1, I_2, ..., I_m}\}$为项目空间
>+ 2.数据集(data)：$D = \{ t_1, t_2,..., t_n \}$为一个事务集（transactions）列表，$D$中每一个事物都是项目空间的子集，即：$t_j \in I, j = 1, 2,...n$
>+ 3.关联规则(association rule)：关联规则具有这种形式 $X => Y$，其中$X, Y \subset I$且$X\cap B = \varnothing$。关联规则中项集$X$和$Y$分别称为这个规则的前件和后件
>+ 4.项集$X$的支持度（support）：项集$X$的支持度（support）定义为数据集$D$中包含$X$的事务在数据集中所占的比例
>+ 5.关联规则的置信度(confidence)：关联规则的置信度(confidence)定义为 $$ {conf(X => Y) = \frac{supp(X\&Y)}{supp(X)}} $$， $ {X\&Y} $表示$ X,Y $一同出现在一条事务中
>+ 6.规则的支持度：规则的支持度定义为$ supp(X => Y) = supp(X\&Y) $  

>关联规则的支持度代表了关联规则的普遍程度，关联规则的置信度测量了该规则的关联强度。


2. 数据表示  
   假设我们有一个用于做机器学习的数据集，这个数据集包含了由预测变量和响应变量组成的样本记录，当预测变量和响应变量都是分类型特征时这个数据集可以转换为事务数据集，从而用于关联规则挖掘。比如一个预测病人是否患癌症的数据集中，有一个属性为细胞分裂，其取值为0, 1, 2,而响应特征即是否患病的取值为是或不是，那么响应特征可以由两个项目来表示，预测特征‘细胞分裂’可以用三个项目来表示。
   
   我们令$n, k$分别表示总的事务数和总的预测项目数，将两个响应项目表示为$I_c, I_{nc}$，这样，我们感兴趣的关联规则前件可以表示为$\{ I_1, I_2,...I_k\}$，后件是$I_c$或$I_{nc}$。

   一种事务的表示方式是二元向量。我们让$J_s = 1, J_s = 0$分别表示项目s是否出现，记$J = (J_1, J_2,...,J_k)$。。每一条事务都是这个二元向量的一个观测。于是，我们感兴趣的关联规则可以划分为两类：$R_c = \{ J => I_c, (J_1, J_2,...J_k) \in \{0, 1\}^k\}$. 和$R_{nc} = \{ J => I_{nc}, (J_1, J_2,...J_k) \in \{0, 1\}^k\}$.


我们希望找到那些具有高置信度的关联规则，然而现在基于约束搜索的算法如Apriori算法等要求用户定义一个最小支持度和最小置信度，这会导致具有高置信度但是支持度很低的关联规则被过滤掉，而我们需要的恰恰是高置信度的的关联规则。这似乎很好理解，在我们判断病人是否患病时，一些症状组合极少出现，但是当它们出现时我们能够判定病人患了这种病，这种病症就是低支持度高置信度的。
现有的算法不能解决这个问题，所以论文提出了一种新的解决方法。

3. 新算法  
首先定义一个新的关联规则测度称为重要性“importance”，形式化定义为：
$$importance(J => I_{\_}) = g_{\_}(J)=f(supp(J => I_{\_}), conf(J => I_{\_}))$$
$I_{\_}$按我们的需要可以选择$I_c,或 I_{nc}$。$f$是用户定义的正的增函数，一旦$f$确定了，我们的目标就变成了寻找$R_c$或$R_{nc}$中最重要的关联规则。
受非贝叶斯优化的启发，我们提出一个定义在规则$R_c$上的概率分布：
$$ P_c(J) = P(J => I_c)= \frac{e^{\xi g(J => I_c)}}{\sum _{allJ}e^{\xi g(J => I_c)}}$$
其中$\xi$是一个可调节参数。  
这里需要注意的一点是这个概率的取值是（0，1）.  
从这个概率分布中我们得知最重要的规则同时也是使得这个概率值最大的规则，这样，如果我们从这个概率分布中随机采样，只要采的的样本足够大，那么最重要的规则近乎以概率1出现在我们的采样中，并且这个规则是样本中出现得最多的那一个。
然而，当k变大时从这个概率分布中随机采样并不容易，因为分母项我们很难计算。为了解决这个问题，引入了Gibbs采样。为了能够使用Gibbs采样方法，我们需要知道给定$J_{-s}$下$J_s$的条件分布，即：
$$
P_c(J_s = 1| J_{-s})=\frac{P_c(J_s=1, J_{-s})}{P_c(J_{-s})}=\frac{P_c(J_s=1, J_{-s})}{P_c(J_s=1, J_{-s}) + P_c(J_s=0, J_{-s})}
$$
    
   $$P_c(J_s = 0| J_{-s})= 1 - P_c(J_s = 1| J_{-s})$$ $J_{-s}$是移除$J_s$后的子向量，$（J_s=1, J_{-s}）$表示将$J_s$放其原来的位置。
于是我们得到从概率分布$P_c(J)$生成$J$的算法：
> (1). 任意选择一个初始向量 $J^{(0)} = (J_1^{(0)},...,J_k^{(0)})$

  > (2). 对$j=1, 2,...M$，从伯努利分布$P_c(J_s|J_1^{(j)},...,J_{s-1}^{(j)},J_{s+1}^{(j-1)},...J_{k}^{(j-1)})$顺序地生成关联规则$J^{(j)}=>I_c$的前件$J^{(j)}$  
  > (3). 返回关联规则样本$(J^{(1)}, J^{(2)},...,J^{(M)})$
  

一旦我们获得了随机采样的样本，那么，最重要的关联规则可以近似地认为是样本中频率最高的关联规则。  
存在这样的可能，即当我们采了一个相对较小的M个随机样本时，每一个关联规则都不相同，每个关联规则的频率都是$\frac{1}{M}$，这说明没有哪个规则是最优的，这时，我们可以计算每一个项目的频率，获得一个频繁项集，在这个项集上应用Apriori算法挖掘最重要的规则。Gibbs采样获得的样本能够大量的减少搜索最重要规则的项目空间，同时能够获得最重要规则的最频繁预测项目。

4. 案例学习  
   假设我们有这样一个事务集：

I\_1 | I\_2 | I\_3 | I\_c 
:-: | :-: | :-: | :-: 
1 | 1 | 0 | 1 
1 | 1| 1 | 0
0 | 1 | 1 | 0
0 | 0 | 1 | 0
1 | 0 | 0 | 1
1 | 1 | 0 | 1


>首先我们选择$J^{(0)}=(1,1,0)$作为初始向量  
>计算第一个分量的条件分布：$$P_c(J_1=1|J_2=1, J_3=0)=\frac{P_c(J_1=1|J_2=1, J_3=0)}{P_c(J_1=1|J_2=1, J_3=0) + P_c(J_1=0|J_2=1, J_3=0)}=
\frac{\frac{e^{\xi g(1, 1, 0 => I_c)}}{\sum _{allJ}e^{\xi g(J => I_c)} }}{\frac{e^{\xi g(1, 1, 0 => I_c)}}{\sum _{allJ}e^{\xi g(J => I_c)} } + \frac{e^{\xi g(0, 1, 0 => I_c)}}{\sum _{allJ}e^{\xi g(J => I_c)} }}=
\frac{e^{\xi g(1, 1, 0 => I_c)}}{e^{\xi g(1, 1, 0 => I_c)} + e^{\xi g(0, 1, 0 => I_c)}}
$$
  我们取：  
  $$g(J=>I_c)=f(supp(J=>I_c), conf(J=>I_c))=supp(J=>I_c) * conf(J=>I_c)$$
  $supp(1, 1, 0 => I_c)= supp(1, 1, 0, 1) = \frac{1}{3}$  
  $conf(1, 1, 0 => I_c)= conf(1, 1, 0, 1) = 1$  
  $ g(1, 1 ,0 => I_c) = \frac{1}{3}$  
  $supp(0, 1, 0 => I_c)= supp(0, 1, 0, 1) = 0$  
  $conf(0, 1, 0 => I_c)= 0$  
  $ g(0, 1 ,0 => I_c) = 0$
  $\frac{e^{\xi g(1, 1, 0 => I_c)}}{e^{\xi g(1, 1, 0 => I_c)} + e^{\xi g(0, 1, 0 => I_c)}}=\frac{e^{\xi \frac{1}{3}}}{e^{\xi \frac{1}{3}} + 1}$  
  如果我们取$\xi = 3$那么$P_c(J_1=1|J_2=1, J_3=0) \approx 0.73$。  
  考虑一种情况即我们在此时取得了$J_1=0$，新向量为$J=(0, 1, 0)$，此时我们发现这条数据并不存在于事务集中，在计算第二个分量时因为(0, 1, 0)和(0, 0, 0)都未曾出现在事务表中，因此$g(0, 1, 0 => I_c)=g(0, 0, 0 => I_c)=0,P_c(J_2=1|J_1=0, J_3=0)=P_c(J_2=0|J_1=0, J_3=0)=0.5$，这里我们发现的一个问题是如果项目空间太大而事务集不足，这个算法一旦依概率分布生成一条不在事务集中的数据，之后可能会以0.5的概率填充所有未出现的事务。此时如果增大$\xi$的值，比如我们的事务集只有两条数据(1, 0, 0, 1),(0, 1, 1, 1),取$\xi = 100$这时采得的样本将会一直重复。


5. 编程实现  
参照论文中的Simulation Studies，example1，我们使用R语言中的MultiOrd包，调用generate.binary()函数，以边缘概率$p=(0.5, 0.5, 0.5, 0.5)$,相关矩阵$$
 R=\begin{matrix}
   1   & 0 & 0 & 0.8 \\
   0   & 1 & 0 & 0  \\
   0   & 0 & 1 & 0.2  \\
   0.8  & 0 & 0.2 & 1
  \end{matrix} 
$$
生成1000个样本。首先分析这一百个样本的情况。

In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [115]:
# 加载数据
filepath = './example1.csv'
df = pd.read_csv(filepath, index_col=0)

In [116]:
print(df.describe())

                V1           V2           V3           V4
count  1000.000000  1000.000000  1000.000000  1000.000000
mean      0.513000     0.512000     0.463000     0.498000
std       0.500081     0.500106     0.498879     0.500246
min       0.000000     0.000000     0.000000     0.000000
25%       0.000000     0.000000     0.000000     0.000000
50%       1.000000     1.000000     0.000000     0.000000
75%       1.000000     1.000000     1.000000     1.000000
max       1.000000     1.000000     1.000000     1.000000


In [117]:
print(df.head())

   V1  V2  V3  V4
1   1   1   1   1
2   1   0   1   1
3   0   0   0   0
4   0   0   1   0
5   1   0   0   1


我们发现数据中存在全为0记录，这并不是我们想要的，另外，(0, 0, 0, 1)这种记录也是无用的，因此，我们需要从数据集中清除掉这些数据。

In [118]:
data = df.values
new_data = []
for i in range(1000):
    if (data[i][:3] == np.array([0, 0, 0])).all():
        continue
    else:
        new_data.append(data[i])

In [119]:
new_data = np.array(new_data)

In [120]:
df = pd.DataFrame(new_data, columns=df.columns)

In [121]:
print(df.head())
print(df.describe())


   V1  V2  V3  V4
0   1   1   1   1
1   1   0   1   1
2   0   0   1   0
3   1   0   0   1
4   1   0   0   1
               V1          V2          V3          V4
count  871.000000  871.000000  871.000000  871.000000
mean     0.588978    0.587830    0.531573    0.571757
std      0.492302    0.492508    0.499289    0.495109
min      0.000000    0.000000    0.000000    0.000000
25%      0.000000    0.000000    0.000000    0.000000
50%      1.000000    1.000000    1.000000    1.000000
75%      1.000000    1.000000    1.000000    1.000000
max      1.000000    1.000000    1.000000    1.000000


这时候我们发现各个项目的均值都增加了，这是必然的，且这对我们的实验没什么影响。现在我们看一下V4=1时的数据分布

In [122]:
print(df[df['V4'] == 1].describe())

               V1          V2          V3     V4
count  498.000000  498.000000  498.000000  498.0
mean     0.903614    0.516064    0.560241    1.0
std      0.295416    0.500244    0.496857    0.0
min      0.000000    0.000000    0.000000    1.0
25%      1.000000    0.000000    0.000000    1.0
50%      1.000000    1.000000    1.000000    1.0
75%      1.000000    1.000000    1.000000    1.0
max      1.000000    1.000000    1.000000    1.0


我们发现在V4也就是我们的Ic=1时，V1=1的频率是0.9,V2=1的频率是0.51,V3=1的频率是0.56，因为V4与V1有很高的的相关系数是0.8。接下来，我们根据论文的算法采样1000个样本，我们预计得到的样本中，V1的频率仍然是最高的。

In [142]:
def supp(s, s_val, vec, data):
    """
    计算supp(Js=1, J_s => Ic)
    """
    count = 0
    vec[s] = s_val
    for i in range(len(data)):
        if (vec == data[i]).all():
            count += 1
    return count / len(data)

In [143]:
def supp_x(s, s_val, vec, data):
    """
    计算规则 J => Ic 的前件 J 的支持度
    """
    vec[s] = s_val
    count = 0
    for i in range(len(data)):
        if (vec[:len(data[i]) - 1] == data[i][:len(data[i]) - 1]).all():
            count += 1
    return count / len(data)

In [144]:
M = 1000
xis = [3, 6, 10]
# xi 取值分别是3， 6， 10，也就是我们采3个样本
data = new_data
for xi in xis:
    init_vec = data[0]
    print(init_vec)
    samples = np.array([init_vec])
    for i in range(M):
        # 根据论文中的算法，采样采的是规则 J => Ic 的前件 J
        for j in range(3):
            """
            有我们之前案例学习的样例中得知，我们需要计算supp(J => Ic), supp(J), conf(J => Ic), g(J => Ic)
            """
            supp1 = supp(j, 1, init_vec, data)
            suppx1 = supp_x(j, 1, init_vec, data)
            conf1 = supp1 / suppx1 if suppx1 != 0 else 0
            g1 = supp1 * conf1
            supp0 = supp(j, 0, init_vec, data)
            suppx0 = supp_x(j, 0, init_vec, data)
            conf0 = supp0 / suppx0 if suppx0 != 0 else 0
            g0 = supp0 * conf0
            p1 = np.exp(xi * g1) / (np.exp(xi * g1) + np. exp(xi * g0))
            p0 = 1 - p1
            init_vec[j] = np.random.choice([1, 0], p=[p1, p0])
        samples = np.row_stack((samples, init_vec))
    samples = np.array(samples)
    samples = pd.DataFrame(samples, columns=df.columns)
    samples.to_csv(f'./samples_of_example1_with_xi{xi}.csv')
    print(f'completed the sampling of xi = {xi}')

[1 1 1 1]
completed the sampling of xi = 3
[1 1 1 1]
completed the sampling of xi = 6
[0 0 1 1]
completed the sampling of xi = 10


已经完成随机样本的采集，接下来我们分析一下随机样本的数据分布。

In [152]:
samples1 = pd.read_csv('./samples_of_example1_with_xi3.csv', index_col=0)
samples2 = pd.read_csv('./samples_of_example1_with_xi6.csv', index_col=0)
samples3 = pd.read_csv('./samples_of_example1_with_xi10.csv', index_col=0)

In [155]:
print(samples1.head())
print(samples2.head())
print(samples3.head())

   V1  V2  V3  V4
0   1   1   1   1
1   0   1   1   1
2   1   0   0   1
3   0   1   1   1
4   1   0   1   1
   V1  V2  V3  V4
0   1   1   1   1
1   1   0   1   1
2   1   1   1   1
3   1   0   1   1
4   1   0   0   1
   V1  V2  V3  V4
0   0   0   1   1
1   0   0   1   1
2   1   1   1   1
3   0   0   0   1
4   0   1   1   1


In [156]:
print(samples1.describe())
print(samples2.describe())
print(samples3.describe())

                V1           V2           V3      V4
count  1001.000000  1001.000000  1001.000000  1001.0
mean      0.596404     0.509491     0.537463     1.0
std       0.490864     0.500160     0.498844     0.0
min       0.000000     0.000000     0.000000     1.0
25%       0.000000     0.000000     0.000000     1.0
50%       1.000000     1.000000     1.000000     1.0
75%       1.000000     1.000000     1.000000     1.0
max       1.000000     1.000000     1.000000     1.0
                V1           V2           V3      V4
count  1001.000000  1001.000000  1001.000000  1001.0
mean      0.654346     0.516484     0.528472     1.0
std       0.475819     0.499978     0.499438     0.0
min       0.000000     0.000000     0.000000     1.0
25%       0.000000     0.000000     0.000000     1.0
50%       1.000000     1.000000     1.000000     1.0
75%       1.000000     1.000000     1.000000     1.0
max       1.000000     1.000000     1.000000     1.0
                V1           V2           V3  

我们发现随着xi取值增大，V1=1的频率也逐渐增大。从概率分布中我们知道，最优的关联规则最有有可能包含项目V1,从我们的原始样本的概率分布和相关矩阵我们能够知道，V1和V4具有很强的关联性。我们继续采论文中example2，example3， example4

In [165]:
for i in [1, 2, 3]:
    data = pd.read_csv(f'./T{i}.csv', index_col=0)
    columns = data.columns
    data = data.values
    M = 1000
    xis = [10, 20, 100, 1000]
    # xi 取值分别是3， 6， 10，也就是我们采3个样本
    for xi in xis:
        init_vec = data[data[:, len(data[0]) - 1] == 1][0]
        print(init_vec)
        samples = np.array([init_vec])
        for j in range(M):
            # 根据论文中的算法，采样采的是规则 J => Ic 的前件 J
            for k in range(len(init_vec) - 1):
                """
                有我们之前案例学习的样例中得知，我们需要计算supp(J => Ic), supp(J), conf(J => Ic), g(J => Ic)
                """
                supp1 = supp(k, 1, init_vec, data)
                suppx1 = supp_x(k, 1, init_vec, data)
                conf1 = supp1 / suppx1 if suppx1 != 0 else 0
                g1 = supp1 * conf1
                supp0 = supp(k, 0, init_vec, data)
                suppx0 = supp_x(k, 0, init_vec, data)
                conf0 = supp0 / suppx0 if suppx0 != 0 else 0
                g0 = supp0 * conf0
                p1 = np.exp(xi * g1) / (np.exp(xi * g1) + np. exp(xi * g0))
                p0 = 1 - p1
                init_vec[k] = np.random.choice([1, 0], p=[p1, p0])
            samples = np.row_stack((samples, init_vec))
        samples = np.array(samples)
        samples = pd.DataFrame(samples, columns=columns)
        samples.to_csv(f'./samples_of_example{i + 1}_with_xi{xi}.csv')
        print(f'completed the sampling of xi = {xi}')

[1 1 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0
 1 0 0 1 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0
 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0
 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 1
 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1]
completed the sampling of xi = 10
[1 1 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0
 1 0 0 1 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 1 0 0

KeyboardInterrupt: 

这样，我们生成了论文中example2和example3的两个随机样本。我们载入生成的随机样本，分析随机样本的数据分布

In [166]:
data = pd.read_csv('./samples_of_example2_with_xi10.csv')

In [167]:
print(data.head())

   Unnamed: 0  V1  V2  V3  V4  V5  V6  V7  V8  V9  ...  V390  V391  V392  \
0           0   1   1   1   1   0   0   1   1   0  ...     0     0     0   
1           1   1   1   1   1   0   1   1   0   1  ...     0     0     1   
2           2   1   1   1   1   0   0   1   1   1  ...     0     0     1   
3           3   0   1   1   1   0   1   1   1   1  ...     1     0     0   
4           4   1   1   1   0   0   0   0   0   0  ...     1     1     0   

   V393  V394  V395  V396  V397  V398  V399  
0     0     0     0     0     0     1     1  
1     0     0     1     0     1     1     1  
2     0     0     0     1     0     0     1  
3     0     0     1     1     0     1     1  
4     1     0     0     1     0     1     1  

[5 rows x 400 columns]


In [168]:
print(data.describe())

        Unnamed: 0           V1           V2           V3           V4  \
count  1001.000000  1001.000000  1001.000000  1001.000000  1001.000000   
mean    500.000000     0.502498     0.522478     0.496503     0.521479   
std     289.108111     0.500244     0.499744     0.500238     0.499788   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%     250.000000     0.000000     0.000000     0.000000     0.000000   
50%     500.000000     1.000000     1.000000     0.000000     1.000000   
75%     750.000000     1.000000     1.000000     1.000000     1.000000   
max    1000.000000     1.000000     1.000000     1.000000     1.000000   

                V5           V6          V7           V8           V9  ...  \
count  1001.000000  1001.000000  1001.00000  1001.000000  1001.000000  ...   
mean      0.487512     0.486513     0.47952     0.528472     0.516484  ...   
std       0.500094     0.500068     0.49983     0.499438     0.499978  ...   
min       0.000000   

第一列是列下标，去掉

In [169]:
data = pd.read_csv('./samples_of_example2_with_xi10.csv', index_col=0)

In [170]:
print(data.describe())

                V1           V2           V3           V4           V5  \
count  1001.000000  1001.000000  1001.000000  1001.000000  1001.000000   
mean      0.502498     0.522478     0.496503     0.521479     0.487512   
std       0.500244     0.499744     0.500238     0.499788     0.500094   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     0.000000     0.000000     0.000000     0.000000   
50%       1.000000     1.000000     0.000000     1.000000     0.000000   
75%       1.000000     1.000000     1.000000     1.000000     1.000000   
max       1.000000     1.000000     1.000000     1.000000     1.000000   

                V6          V7           V8           V9          V10  ...  \
count  1001.000000  1001.00000  1001.000000  1001.000000  1001.000000  ...   
mean      0.486513     0.47952     0.528472     0.516484     0.490509  ...   
std       0.500068     0.49983     0.499438     0.499978     0.500160  ...   
min       0.000000   

我们画出均值折线图。另外因为最后一列是规则后件即我们的Ic,在分析中并不需要，因此我们将最后一列删除。

In [180]:
data = data.drop('V399', 1)

In [181]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V389,V390,V391,V392,V393,V394,V395,V396,V397,V398
0,1,1,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,1,1,0,1,1,0,1,0,...,0,0,0,1,0,0,1,0,1,1
2,1,1,1,1,0,0,1,1,1,1,...,0,0,0,1,0,0,0,1,0,0
3,0,1,1,1,0,1,1,1,1,1,...,1,1,0,0,0,0,1,1,0,1
4,1,1,1,0,0,0,0,0,0,0,...,0,1,1,0,1,0,0,1,0,1


In [182]:
data.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V389,V390,V391,V392,V393,V394,V395,V396,V397,V398
count,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.00000,1001.000000,1001.000000,1001.000000,...,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000
mean,0.502498,0.522478,0.496503,0.521479,0.487512,0.486513,0.47952,0.528472,0.516484,0.490509,...,0.488511,0.483516,0.525475,0.480519,0.516484,0.522478,0.488511,0.541459,0.476523,0.492507
std,0.500244,0.499744,0.500238,0.499788,0.500094,0.500068,0.49983,0.499438,0.499978,0.500160,...,0.500118,0.499978,0.499600,0.499870,0.499978,0.499744,0.500118,0.498527,0.499698,0.500194
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.00000,1.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [183]:
print(data.mean().max())
print(data.mean().min())

0.5544455544455544
0.44555444555444557


我们发现样本中出现的最多的项目的频率是0.55，最低是0.44，我们继续载入之后的随机样本。

In [186]:
data1 = pd.read_csv('./samples_of_example2_with_xi20.csv', index_col=0).drop('V399', 1)
data2 = pd.read_csv('./samples_of_example2_with_xi100.csv', index_col=0).drop('V399', 1)
data3 = pd.read_csv('./samples_of_example2_with_xi1000.csv', index_col=0).drop('V399', 1)

In [187]:
data1.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V389,V390,V391,V392,V393,V394,V395,V396,V397,V398
count,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.00000,1001.000000,1001.000000,...,1001.000000,1001.00000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.00000
mean,0.495504,0.494505,0.508492,0.508492,0.501499,0.511489,0.475524,0.48951,0.466533,0.477522,...,0.512488,0.51049,0.505495,0.504496,0.474525,0.482517,0.503497,0.494505,0.504496,0.50050
std,0.500230,0.500220,0.500178,0.500178,0.500248,0.500118,0.499650,0.50014,0.499128,0.499744,...,0.500094,0.50014,0.500220,0.500230,0.499600,0.499944,0.500238,0.500220,0.500230,0.50025
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.00000,0.000000,0.000000,...,1.000000,1.00000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.00000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,...,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,...,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000


In [188]:
data2.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V389,V390,V391,V392,V393,V394,V395,V396,V397,V398
count,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,...,1001.000000,1001.000000,1001.000000,1001.00000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000
mean,0.522478,0.495504,0.522478,0.496503,0.484515,0.511489,0.506494,0.508492,0.503497,0.508492,...,0.501499,0.486513,0.517483,0.48951,0.466533,0.495504,0.490509,0.497502,0.521479,0.506494
std,0.499744,0.500230,0.499744,0.500238,0.500010,0.500118,0.500208,0.500178,0.500238,0.500178,...,0.500248,0.500068,0.499944,0.50014,0.499128,0.500230,0.500160,0.500244,0.499788,0.500208
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [189]:
data3.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V389,V390,V391,V392,V393,V394,V395,V396,V397,V398
count,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.00000,...,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.00000,1001.000000,1001.000000,1001.000000,1001.000000
mean,0.471528,0.517483,0.482517,0.507493,0.485514,0.471528,0.501499,0.492507,0.478521,0.49950,...,0.484515,0.507493,0.522478,0.492507,0.488511,0.46953,0.481518,0.474525,0.476523,0.488511
std,0.499438,0.499944,0.499944,0.500194,0.500040,0.499438,0.500248,0.500194,0.499788,0.50025,...,0.500010,0.500194,0.499744,0.500194,0.500118,0.49932,0.499908,0.499600,0.499698,0.500118
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.00000,...,0.000000,1.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000


In [190]:
print('the max mean of data1', data1.mean().max())
print('the min mean of data1', data1.mean().min())
print('the max mean of data2', data2.mean().max())
print('the min mean of data2', data2.mean().min())
print('the max mean of data3', data3.mean().max())
print('the min mean of data3', data3.mean().min())

the max mean of data1 0.5384615384615384
the min mean of data1 0.4565434565434565
the max mean of data2 0.5514485514485514
the min mean of data2 0.4515484515484515
the max mean of data3 0.5464535464535465
the min mean of data3 0.45854145854145856


我们发现四个样本的出现最大的项目其频率都没有超过0.6，并且，V1, V2, V3在样本中出现的频率也不高。这似乎和我们预期的不一样，我们希望的结果是，V1, V2, V3出现的频率应该是最高的，因为先验分布中这三个项目和Ic(V399)具有很强的相关性。
我们载入第三个example的随机样本，看看是否也是如此。

In [191]:
data1 = pd.read_csv('./samples_of_example3_with_xi20.csv', index_col=0).drop('V399', 1)
data2 = pd.read_csv('./samples_of_example3_with_xi100.csv', index_col=0).drop('V399', 1)
data3 = pd.read_csv('./samples_of_example3_with_xi1000.csv', index_col=0).drop('V399', 1)
data4 = pd.read_csv('./samples_of_example3_with_xi10.csv', index_col=0).drop('V399', 1)

In [192]:
data1.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V389,V390,V391,V392,V393,V394,V395,V396,V397,V398
count,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,...,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000
mean,0.486513,0.496503,0.512488,0.513487,0.484515,0.480519,0.502498,0.481518,0.506494,0.473526,...,0.509491,0.471528,0.513487,0.525475,0.498501,0.480519,0.515485,0.481518,0.515485,0.497502
std,0.500068,0.500238,0.500094,0.500068,0.500010,0.499870,0.500244,0.499908,0.500208,0.499548,...,0.500160,0.499438,0.500068,0.499600,0.500248,0.499870,0.500010,0.499908,0.500010,0.500244
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [193]:
print('the max mean of data1', data1.mean().max())
print('the min mean of data1', data1.mean().min())
print('the max mean of data2', data2.mean().max())
print('the min mean of data2', data2.mean().min())
print('the max mean of data3', data3.mean().max())
print('the min mean of data3', data3.mean().min())
print('the max mean of data4', data4.mean().max())
print('the min mean of data4', data4.mean().min())

the max mean of data1 0.5434565434565435
the min mean of data1 0.44555444555444557
the max mean of data2 0.5404595404595405
the min mean of data2 0.44555444555444557
the max mean of data3 0.5394605394605395
the min mean of data3 0.45754245754245754
the max mean of data4 0.5534465534465535
the min mean of data4 0.45354645354645357


这个结果说明我们的采样出现了问题。我们检查一下我们根据概率分布和相关矩阵生成的数据T1和T2

In [194]:
T1 = pd.read_csv('./T1.csv', index_col=0)
T2 = pd.read_csv('./T2.csv', index_col=0)

In [195]:
T1.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V390,V391,V392,V393,V394,V395,V396,V397,V398,V399
1,1,1,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,1
2,1,1,1,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,1,0,1
3,1,1,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,1
4,1,1,1,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,1
5,0,0,0,1,1,0,0,0,0,0,...,1,1,0,0,1,1,0,0,0,0


In [196]:
T2.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V390,V391,V392,V393,V394,V395,V396,V397,V398,V399
1,1,1,1,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,1
2,1,1,1,1,1,1,1,1,0,0,...,1,0,1,0,0,1,0,0,0,1
3,1,1,1,1,0,1,1,1,0,1,...,0,1,0,1,0,0,0,0,0,1
4,1,1,1,1,1,0,0,1,0,1,...,0,0,0,1,0,0,0,0,1,1
5,1,1,1,1,1,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,1


In [197]:
T1.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V390,V391,V392,V393,V394,V395,V396,V397,V398,V399
count,1000.000000,1000.000000,1000.000000,1000.0000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.00000,1000.000000,1000.0000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.811000,0.810000,0.812000,0.2000,0.194000,0.220000,0.186000,0.233000,0.202000,0.194000,...,0.215000,0.20800,0.207000,0.2000,0.201000,0.196000,0.195000,0.217000,0.205000,0.811000
std,0.391705,0.392497,0.390908,0.4002,0.395627,0.414454,0.389301,0.422954,0.401693,0.395627,...,0.411028,0.40608,0.405358,0.4002,0.400949,0.397167,0.396399,0.412409,0.403904,0.391705
min,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,1.000000,1.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.00000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [198]:
T2.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V390,V391,V392,V393,V394,V395,V396,V397,V398,V399
count,1000.00000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000
mean,0.81700,0.815000,0.815000,0.488000,0.52000,0.519000,0.503000,0.499000,0.511000,0.509000,...,0.205000,0.217000,0.191000,0.189000,0.203000,0.190000,0.199000,0.205000,0.19200,0.816000
std,0.38686,0.388492,0.388492,0.500106,0.49985,0.499889,0.500241,0.500249,0.500129,0.500169,...,0.403904,0.412409,0.393286,0.391705,0.402434,0.392497,0.399448,0.403904,0.39407,0.387678
min,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,1.00000,1.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1.000000
50%,1.00000,1.000000,1.000000,0.000000,1.00000,1.000000,1.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1.000000
75%,1.00000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1.000000
max,1.00000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


我们分析所有Ic=1的数据

In [204]:
data1 = T1[T1['V399'] == 1]
data2 = T2[T2['V399'] == 1]

In [205]:
data1.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V390,V391,V392,V393,V394,V395,V396,V397,V398,V399
1,1,1,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,1
2,1,1,1,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,1,0,1
3,1,1,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,1
4,1,1,1,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,1
7,1,1,1,0,0,0,0,0,0,0,...,1,0,0,0,0,1,1,0,0,1


In [206]:
data2.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V390,V391,V392,V393,V394,V395,V396,V397,V398,V399
1,1,1,1,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,1
2,1,1,1,1,1,1,1,1,0,0,...,1,0,1,0,0,1,0,0,0,1
3,1,1,1,1,0,1,1,1,0,1,...,0,1,0,1,0,0,0,0,0,1
4,1,1,1,1,1,0,0,1,0,1,...,0,0,0,1,0,0,0,0,1,1
5,1,1,1,1,1,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,1


In [207]:
data1.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V390,V391,V392,V393,V394,V395,V396,V397,V398,V399
count,811.000000,811.000000,811.000000,811.000000,811.000000,811.000000,811.000000,811.000000,811.000000,811.000000,...,811.000000,811.000000,811.000000,811.000000,811.000000,811.000000,811.000000,811.000000,811.000000,811.0
mean,0.997534,0.996301,0.996301,0.194821,0.198520,0.231813,0.176326,0.239211,0.203453,0.191122,...,0.220715,0.208385,0.204686,0.196054,0.192355,0.198520,0.187423,0.217016,0.203453,1.0
std,0.049629,0.060745,0.060745,0.396307,0.399132,0.422250,0.381332,0.426865,0.402815,0.393428,...,0.414985,0.406404,0.403721,0.397255,0.394394,0.399132,0.390491,0.412468,0.402815,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
50%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
75%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0


In [208]:
data2.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V390,V391,V392,V393,V394,V395,V396,V397,V398,V399
count,816.000000,816.000000,816.000000,816.000000,816.000000,816.000000,816.000000,816.000000,816.000000,816.000000,...,816.000000,816.000000,816.000000,816.000000,816.000000,816.000000,816.000000,816.000000,816.000000,816.0
mean,0.998775,0.997549,0.997549,0.496324,0.513480,0.520833,0.490196,0.504902,0.519608,0.512255,...,0.199755,0.214461,0.177696,0.194853,0.203431,0.192402,0.199755,0.210784,0.191176,1.0
std,0.035007,0.049477,0.049477,0.500293,0.500125,0.499872,0.500210,0.500283,0.499922,0.500156,...,0.400061,0.410699,0.382491,0.396330,0.402798,0.394429,0.400061,0.408116,0.393469,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
50%,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0


在我们的数据T1, T2中，Ic=1时前三项出现的频率都是最高的。
回想到我们在案例学习中的分析，当采样时生成了一条不在数据集中的数据时，计算向量J的下一个分量时，由于其没有出现过，所以支持度和置信度都为0，取得0和1的概率都是0.5，原因是，假设我们有10个项目，每个项目取值都是0或1，那么，我们最少需要$2^{10}$条数据才能保证所有的项目组合都存在.为了确定是不是这个原因，我们继续做一个案例分析。

I\_1 | I\_2 | I\_3 | I\_4 |I\_5 | I\_6 | I\_7 | I\_c 
:-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: 
1 | 1 | 0 | 1 | 1 | 1 | 0 | 1 
1 | 1 | 1 | 0 | 0 | 1 | 1 | 1 
0 | 1 | 1 | 0 | 1 | 0 | 1 | 1 
0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 
1 | 0 | 0 | 1 | 1 | 0 | 1 | 1 
1 | 1 | 0 | 1 | 0 | 1 | 0 | 0 

我们选择第一个向量（1， 1， 0， 1， 1， 1，0，1）作为初始向量$J^{(0)}$。
生成$J^{(1)}$:
>+ $supp(J_1 = 1, J_{-1} => I_c = 1)=\frac{1}{6}$
>+ $supp(J_1 = 1, J_{-1})=\frac{1}{6}$
>+ $conf(J_1 = 1, J_{-1} => I_c = 1)=1$
>+ $g(J_1 = 1, J_{-1} => I_c = 1) = f(supp, conf)= supp * conf = \frac{1}{6}$
>+ $supp(J_1 = 0, J_{-1} => I_c = 1)=0$
>+ $supp(J_1 = 0, J_{-1})=0$
>+ $conf(J_1 = 0, J_{-1} => I_c = 1)=0$
>+ $g(J_1 = 0, J_{-1} => I_c = 1) = f(supp, conf)= supp * conf = 0$
我们取$\xi = 10$
>+ $P1 = \frac{e^{\xi \frac{1}{6}}}{e^{\xi \frac{1}{6}} + 1} \approx 0.84$
>+ $P0 = 1 - P1 = 0.16$

假设我们取得了$J_1^{(1)}=0$，那么新的向量变为：（0， 1， 0， 1， 1， 1，0，1），接下来计算第二个分量$J_2^{(1)}$

>+ $supp(J_2 = 1, J_{-2} => I_c = 1)=0$
>+ $supp(J_2 = 1, J_{-2})=0$
>+ $conf(J_2 = 1, J_{-2} => I_c = 1)=0$
>+ $g(J_2 = 1, J_{-2} => I_c = 1) = f(supp, conf)= supp * conf = 0$
>+ $supp(J_2 = 0, J_{-2} => I_c = 1)=0$
>+ $supp(J_2 = 0, J_{-2})=0$
>+ $conf(J_2 = 0, J_{-2} => I_c = 1)=0$
>+ $g(J_2 = 0, J_{-2} => I_c = 1) = f(supp, conf)= supp * conf = 0$
我们取$\xi = 10$
>+ $P1 = \frac{e^{\xi 0}}{e^{\xi 0} + e^{\xi 0}}= 0.5$
>+ $P0 = 1 - P1 = 0.5$

我们发现因为这个数据没有在我们的数据集中出现，所以之后的每个分量的概率都是0.5.
问题是，是不是使用该方法不能处理项目空间大而数据集小的情况呢？还是我们理解错了呢？

想了一下，或许我们可以尝试另外一种做法，即不使用具体的向量，我们在计算分量的时候不比较所有的数据，我们只比较这个分量。

In [214]:
T1 = pd.read_csv('./T1.csv', index_col=0)

In [215]:
init_vec = T1[T1['V399'] == 1].iloc[0]
init_vec

V1      1
V2      1
V3      1
V4      1
V5      0
       ..
V395    0
V396    0
V397    0
V398    1
V399    1
Name: 1, Length: 399, dtype: int64

In [216]:
def supp(s, s_val, data):
    """
    计算Js = s_val 时 Ic = 1的频率即关联规则支持度
    """
    count = 0
    for i in range(len(data)):
        if data[i][s] == s_val and data[i][len(data[i]) - 1] == 1:
            count += 1
    return count / len(data)

def supp_x(s, s_val, data):
    """
    计算s = s_val出现的频率即前件支持度
    """
    count = 0
    for i in range(len(data)):
        if data[i][s] == s_val:
            count += 1
    return count / len(data)

In [223]:
xi = 10
M = 300
data = T1.values
samples = np.array(init_vec)
for i in range(M):
    for j in range(len(init_vec) - 1):
        supp1 = supp(j, 1, data)
        suppx1 = supp_x(j, 1, data)
        conf1 = supp1 / suppx1
        supp0 = supp(j, 0, data)
        suppx0 = supp_x(j, 0, data)
        conf0 = supp0 / suppx0
        g1 = supp1 * conf1
        g0 = supp0 * conf0
        p1 = np.exp(xi * g1) / (np.exp(xi * g1) + np.exp(xi * g0))
        init_vec[j] = np.random.choice([1, 0], p=[p1, 1 - p1])
    samples = np.row_stack((samples, init_vec))
samples = pd.DataFrame(samples, columns=T1.columns)
samples.to_csv('./new_samples_of_T1_with_xi10')

In [224]:
print(samples.head())

   V1  V2  V3  V4  V5  V6  V7  V8  V9  V10  ...  V390  V391  V392  V393  V394  \
0   1   1   1   0   0   1   0   0   0    0  ...     0     0     0     0     0   
1   1   1   1   0   0   0   0   0   0    0  ...     0     0     0     0     0   
2   1   1   1   0   0   0   0   0   0    0  ...     0     0     0     0     0   
3   1   1   1   0   0   0   0   0   0    0  ...     0     0     0     0     0   
4   1   1   1   0   0   0   0   0   0    0  ...     1     1     0     0     0   

   V395  V396  V397  V398  V399  
0     0     0     0     0     1  
1     0     0     0     0     1  
2     0     0     0     0     1  
3     0     0     0     0     1  
4     0     0     0     0     1  

[5 rows x 399 columns]


In [225]:
print(samples.describe())

               V1     V2     V3          V4          V5          V6  \
count  301.000000  301.0  301.0  301.000000  301.000000  301.000000   
mean     0.996678    1.0    1.0    0.009967    0.009967    0.033223   
std      0.057639    0.0    0.0    0.099500    0.099500    0.179516   
min      0.000000    1.0    1.0    0.000000    0.000000    0.000000   
25%      1.000000    1.0    1.0    0.000000    0.000000    0.000000   
50%      1.000000    1.0    1.0    0.000000    0.000000    0.000000   
75%      1.000000    1.0    1.0    0.000000    0.000000    0.000000   
max      1.000000    1.0    1.0    1.000000    1.000000    1.000000   

               V7          V8          V9         V10  ...        V390  \
count  301.000000  301.000000  301.000000  301.000000  ...  301.000000   
mean     0.013289    0.029900    0.026578    0.013289  ...    0.019934   
std      0.114700    0.170596    0.161115    0.114700  ...    0.140005   
min      0.000000    0.000000    0.000000    0.000000  ...    0.

我们发现，新的样本中，前三项出现的概率几乎都是1。这是我们想要的结果，但是，这个线下能否用Gibbs采样理论来解释呢？